# Lab 07

## Zadanie 1

Przygotuj demo programu znajdującego miejsca zerowe metodą Newtona. Wykorzystując `argparse` ([link](https://docs.python.org/3.6/library/argparse.html#module-argparse)) lub `optparse` ([link](https://docs.python.org/3.6/library/optparse.html)) obsłuż:
- ustalane punktu startowego,
- wielkość kroku w pochodnej,           <- niepotrzebne, gdy używamy gotowej metody do obliczania pochodnej*
- ilość kroków metody,
- dokładność
- pomoc

Program uruchamiamy podając, np.:

`./newton.py x**2+x+1 -h 0.00001`

In [1]:
class TestArgs:
  func = "x**2-7*x+1"         # 0.146, 6.854
  start = 5
  n_steps = 10
  precision = 0.0001

In [2]:
from scipy.misc import derivative
import argparse
import re
import warnings

# czyszczę warning ze względu na to, że funkcja derivative z scipy jest przestarzała (a jeśli dobrze zrozumiałam,
# to tylko ona została zezwolona do obliczenia pochodnej - stąd decyzja o nieużywaniu niczego "nowszego")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

def f(x):
    # regex w razie, gdyby wpisana funkcja zależała od innej zmiennej niż x
    # zakładam tak jak w przykładzie, że funkcja zostaje wpisana po "pythonowemu" (tzn. np potęga to '**')
    func = re.sub("[a-zA-z]+", str(x), args.func)       
    return eval(func)

parser = argparse.ArgumentParser()
parser.add_argument("func", type=str, help="Funkcja, dla której będzie wyznaczane przybliżenie miejsca zerowego metodą Newtona")
parser.add_argument("start", type=float, help="Punkt startowy")
parser.add_argument("n_steps", type=int, help="Ilość kroków metody; domyślnie=10", default=10, nargs='?')
parser.add_argument("precision", type=float, help="Dokładność; domyślnie=0.0001", default=0.0001, nargs='?')
#args = parser.parse_args()         # <- gdy skrypt uruchamiany jest bezpośrenio z terminala
args = TestArgs()                   # <- na potrzeby użycia skryptu w jupyterze

result = args.start

for _ in range(args.n_steps):
    prev = result
    result = prev - f(prev)/derivative(f, prev)

    if abs(result - prev) < args.precision or abs(f(result)) < args.precision:
        break

print("Przybliżenie miejsca zerowego: ", result)

Przybliżenie miejsca zerowego:  6.854103343465046


## Zadanie 2

Zadanie należy wykonać wykorzystując program `BagOfWords` z poprzednich zajęć.


Przerób ją tak, żeby interpunkcja, cyfry i wszelkie inne znaki nie przeszkadzały w parsowaniu tekstu. Uruchom ja na [tekście hamleta](http://www.gutenberg.org/cache/epub/1787/pg1787.txt). Ile razy występuje słowo hamlet? Jak brzmi 10 najczęściej występujących słów?

In [3]:
import re
import json
from collections import defaultdict
import operator

'''
W rozwiązaniu wszelkiego typu skróty (słowa z apostrofem w środku np. it's, we'll, hamlet's) traktowane są jako odrębne słowo 
(jak na maturze :p), z uwagi na ich niejedoznaczność - niewiadomo jak należałoby je traktować: czy rozwijać na podstawie najbardziej
popularnych np. it's na it + is i zliczać oba słowa osobno, czy tylko zliczać pierwszy człon, ponadto w tekście niektóre litery w słowach
również zastępowane są apostrofami (zatem nie jest to konkretny skrót, z reguły jest to litera 'e')
'''

class BagOfWords:
    def __init__(self, text=""):
        if isinstance(text, dict):
            self.dict = text.copy()
            return 

        self.dict = defaultdict(int)

        if isinstance(text, str):
            self.add_text_to_bag(text)
        else:
            with text as doc:
                for line in doc:
                    self.add_text_to_bag(line)

    def add_text_to_bag(self, text):
        match = re.findall("([a-zA-Z]+(['-][a-zA-Z'-]+)+|[a-zA-Z]+)", text) 
        
        for group in match:
                word = group[0]
                self.dict[word.lower()] += 1

    def get_sorted_dict(self):
        return sorted(self.dict.items(), key=operator.itemgetter(1), reverse=True)

    def get_top(self, n):
        return ', '.join('{}:{}'.format(key, value) for key, value in self.get_sorted_dict()[:n])

    def __str__(self):
        return ', '.join('{}:{}'.format(key, value) for key, value in self.get_sorted_dict())

    def __contains__(self, word):
        return word in self.dict

    def __iter__(self):
        return (key for key, val in self.get_sorted_dict())

    def __add__(self, other):
        result = self.dict.copy()

        for key, val in other.dict.items():
            if (key in result):
                result[key] += other.dict[key]
            else:
                result[key] = other.dict[key]

        return BagOfWords(result)

    def __getitem__(self, arg):
        return self.dict[arg.lower()]

    def __setitem__(self, arg, val):
        self.dict[arg.lower()] = val

    def save(self, file="bag_of_words.json"):
        with open(file, "w") as save_file:
            json.dump(self.dict, save_file)

    def load(self, file="bag_of_words.json"):
        with open(file) as load_file:
            self.dict = json.load(load_file)


In [4]:
bow = BagOfWords(open("hamlet.txt"))
print(bow["hamlet"])


104


In [5]:
print(bow.get_top(10))

the:1087, and:963, to:735, of:669, i:575, you:555, a:554, my:520, in:434, it:419


## Zadanie 3

Wykorzystując `pickle` zapisz i odczytaj klasy z poprzedniego zadania nakarmonej Hamletem. Porównaj metody i rozmiar.

In [6]:
import pickle
import os
import json

bow_loaded = None

In [7]:
%%timeit

with open('bow.pkl', 'wb') as file:
    pickle.dump(bow, file)

4.43 ms ± 517 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
file_size = os.path.getsize('bow.pkl')
print(f"Pickle file size: {file_size} bytes")

Pickle file size: 55399 bytes


In [9]:
%%timeit

with open('bow.pkl', 'rb') as file:
    bow_loaded = pickle.load(file)

3.85 ms ± 940 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
with open('bow.pkl', 'rb') as file:
    bow_loaded = pickle.load(file)

print(bow_loaded.get_top(10))

the:1087, and:963, to:735, of:669, i:575, you:555, a:554, my:520, in:434, it:419


In [11]:
%%timeit

bow.save()

42.9 ms ± 3.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
file_size = os.path.getsize('bag_of_words.json')
print(f"JSCN file size: {file_size} bytes")

JSCN file size: 65355 bytes


In [13]:
%%timeit

bow.load()

5.53 ms ± 754 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Porównując zapis i odczyt obiektu bow do pliku za pomocą dwóch modułów: pickle i json, możemy stwierdzić, że pickle działa zdecydowanie szybciej (zapis ~10x, odczyt ~2x), ponadto plik zapisany przy użyciu pickle ma mniejszy rozmiar niż json (o ok. 10Mb)


## Zadanie 4

Wykorzystując [https://gist.github.com/pamelafox/986163](https://gist.github.com/pamelafox/986163) podaj aktualną godzinę we:
- wszystkich krajach, wyświetlając je zgrupowane względem kontynentów,
- przeprowadź symulacyjne wyświetlanie kolejnych krajów w miarę jak w danym kraju wybija północ, opóżnienie wyświetlania ustaw proporcjonalne do realnego czasu

In [14]:
from countryinfo import countries
from datetime import datetime
from dateutil import tz
from itertools import groupby
from operator import itemgetter
from time import sleep

In [15]:
data = sorted(countries, key=itemgetter("continent"))

for continent, val in groupby(data, key=itemgetter("continent")):
    print(continent.upper().center(41))
    print("-------------------------------------------")
    for country in val:
        timezone = tz.gettz(country["timezones"][0])
        time = datetime.now(tz=timezone).strftime("%H:%M:%S")
        print("{0:35}{1}".format(country["name"], time))
    print()


                  AFRICA                 
-------------------------------------------
Angola                             23:50:21
Burkina Faso                       22:50:21
Burundi                            00:50:21
Benin                              23:50:21
Botswana                           00:50:21
Democratic Republic of the Congo   23:50:21
Republic of the Congo              23:50:21
CÃ´te d'Ivoire                     22:50:21
Cameroon                           23:50:21
Cape Verde                         21:50:21
Djibouti                           01:50:21
Egypt                              00:50:21
Eritrea                            01:50:21
Ethiopia                           01:50:21
Gabon                              23:50:21
Ghana                              22:50:21
The Gambia                         22:50:21
Guinea                             22:50:21
Guinea-Bissau                      22:50:21
Kenya                              01:50:21
Liberia                           

In [16]:
offsets_to_midnight = {}

for country in countries:
    timezone = tz.gettz(country["timezones"][0])
    time = datetime.now(tz=timezone)
    offset = time.utcoffset() 
    offsets_to_midnight[country["name"]] = offset if offset.days >= 0 else -offset

data = sorted(offsets_to_midnight.items(), key=lambda kv: kv[1])

prev_offset = 0
delay_base = 2

for off, val in groupby(data, lambda kv: kv[1]):
    sleep(delay_base * (off.seconds - prev_offset)/3600)
    prev_offset = off.seconds

    for country in val:
        print(country[0])

Burkina Faso
CÃ´te d'Ivoire
Ghana
The Gambia
Guinea
Guinea-Bissau
Republic of Ireland
Iceland
Liberia
Mali
Mauritania
Portugal
Sierra Leone
Senegal
SÃ£o TomÃ© and PrÃ­ncipe
Togo
United Kingdom


Andorra
Albania
Angola
Austria
Belgium
Benin
Democratic Republic of the Congo
Republic of the Congo
Cameroon
Cape Verde
Czech Republic
Germany
Denmark
France
Gabon
Hungary
Italy
Liechtenstein
Luxembourg
Macedonia
Malta
Niger
Nigeria
Kingdom of the Netherlands
Norway
Poland
Sweden
Slovenia
Slovakia
San Marino
Tunisia
Vatican City
Algeria
Bosnia and Herzegovina
Central African Republic
Chad
Croatia
Equatorial Guinea
Monaco
Montenegro
Morocco
Serbia
Spain
Switzerland
Bulgaria
Burundi
Brazil
Botswana
Cyprus
Estonia
Egypt
Finland
Greece
Israel
Jordan
Lebanon
Lesotho
Lithuania
Latvia
Libya
Malawi
Mozambique
Namibia
Romania
Russia
Rwanda
Sudan
Syria
Ukraine
Zambia
Zimbabwe
Moldova
South Africa
Swaziland
Argentina
Bahrain
Belarus
Chile
Djibouti
Eritrea
Ethiopia
Iraq
Kenya
Kuwait
Madagascar
Paraguay
Qatar
Saudi Arabia
Somalia
Suriname
Turkey
Tanzania
Uganda
Uruguay
Yemen
Comoros
Canada
Iran
Antigua and Barbuda
Armenia
Azerbaijan
Barbados
Bolivia
Dominica
Dominican Republic
Georgia
Guyana
Maurit

## Zadanie 5

Dla klasy `BagOfWords` napisz metody `save` oraz `load` wykorzystujące `json`'a do zapisu i odczytu danych.

In [17]:
#implementacje metod save oraz load wyżej, przy zadaniu nr 2

bow = BagOfWords("ala ma kota ala ma ala")
print(bow)

ala:3, ma:2, kota:1


In [18]:
bow.save()
with open("bag_of_words.json") as file:
    print(list(file)[0])

{"ala": 3, "ma": 2, "kota": 1}


In [19]:
bow2 = BagOfWords()
print(bow2.__str__() == "")

bow2.load("bag_of_words.json")
print(bow2)

True
ala:3, ma:2, kota:1
